In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
books = pd.read_csv(f"{path}/books.csv")

In [ ]:
books.head()

In [ ]:
books.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
ax = plt.axes()
sns.heatmap(books.isna(),cbar=False,ax=ax)

plt.xlabel("Columns")
plt.ylabel("Missing Values")
plt.show()

In [ ]:
import numpy as np

books["missing_description"] = np.where(books["description"].isna(),1,0)
books["age_of_book"] = 2025-books["published_year"]

In [ ]:
columns_of_interest = ["num_pages","age_of_book","missing_description","average_rating"]
correlation_matrix = books[columns_of_interest].corr(method= "spearman")
sns.set_theme(style="white")
plt.figure(figsize=(8,6))
heatmap = sns.heatmap(correlation_matrix,annot=True,fmt=".2f",cmap="coolwarm",cbar_kws={"label": "Spearman Correlation"})
heatmap.set_title("Correlation heatmap")
plt.show()

In [ ]:
books_missing = books[(~books["description"].isna()) &
      (~books["average_rating"].isna()) &
      (~books["num_pages"].isna()) &
      (~books["published_year"].isna())
]

In [ ]:
books_missing

In [ ]:
books_missing["categories"].value_counts().reset_index().sort_values("count",ascending=False)

In [ ]:
import numpy as np

books_missing.loc[:, "words_in_description"] = books_missing["description"].str.split().str.len()
books_missing.loc[:, "age_of_book"] = 2025-books_missing["published_year"]

In [ ]:
books_missing_25_words = books_missing[books_missing["words_in_description"] > 25]

In [ ]:
books_missing.describe()

In [ ]:
books_missing_25_words.describe()

In [ ]:
books_missing_25_words.loc[:,"title_and_subtite"]=(
    np.where(books_missing_25_words["subtitle"].isna(),
             books_missing_25_words["title"],
             books_missing_25_words[["title","subtitle"]].astype(str).agg(": ".join,axis=1))
)

In [ ]:
books_missing_25_words

In [ ]:
books_missing_25_words.loc[:,"tagged_description"] = books_missing_25_words[["isbn13","description"]].astype(str).agg(" ".join,axis=1)

In [ ]:
books_missing_25_words

In [ ]:
(
    books_missing_25_words
    .drop(["subtitle","missing_description","age_of_book","words_in_description"],axis=1)
    .to_csv("books_cleaned.csv",index=False)
)

In [ ]:
books=books_missing_25_words

In [ ]:
books["categories"].value_counts().reset_index().sort_values("count",ascending=False)

In [ ]:
books["categories"].value_counts().reset_index().query("count > 50")

In [ ]:
category_mapping = {'Fiction' : "Fiction",
                    'Juvenile Fiction': "Children's Fiction",
                    'Biography & Autobiography': "Nonfiction" ,
                    'History' : "Nonfiction",
                    'Literary Criticism': "Nonfiction",
                    'Philosophy': "Nonfiction",
                    'Religion': "Nonfiction",
                    'Comics & Graphic Novels': "Fiction",
                    'Drama': "Fiction",
                    'Science': "Nonfiction",
                    'Juveline NonFiction' :"Children's Nonfiction",
                    'Poetry':"Fiction"}

books["simple_categories"] = books["categories"].map(category_mapping)


In [ ]:
books['simple_categories']

In [26]:
!pip install gliclass
!pip install -U transformers>=4.48.0


  Using cached transformers-4.48.2-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.48.2-py3-none-any.whl (9.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gliclass 0.1.11 requires transformers<=4.48.2,>=4.37.2, but you have transformers 4.52.4 which is incompatible.


#### Now NaN will be converted to categories using zero shot classification

In [27]:
from gliclass import GLiClassModel, ZeroShotClassificationPipeline
from transformers import AutoTokenizer

model = GLiClassModel.from_pretrained("knowledgator/gliclass-modern-base-v2.0-init")
tokenizer = AutoTokenizer.from_pretrained("knowledgator/gliclass-modern-base-v2.0-init", add_prefix_space=True)
classifier = ZeroShotClassificationPipeline(model, tokenizer, classification_type='multi-label', device='cuda:0')

# from transformers import pipeline

fiction_classifier =["Fiction","NonFiction"]

# classifier = pipeline("zero-shot-classification",
#                       model="facebook/bart-large-mnli",
#                       device="cuda")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/606M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [28]:
sequence = books.loc[books["simple_categories"]=="Fiction","description"].reset_index(drop=True)[0]

In [29]:
classifier(sequence,fiction_classifier)

100%|██████████| 1/1 [00:02<00:00,  2.74s/it]


[[{'label': 'Fiction', 'score': 0.8300336599349976}]]

In [30]:
import numpy as np
predictions = classifier(sequence,fiction_classifier)[0]
max_index = np.argmax([p['score'] for p in predictions])
max_label = predictions[max_index]["label"]
max_label

100%|██████████| 1/1 [00:03<00:00,  3.01s/it]


'Fiction'

In [31]:
import numpy as np

def generate_predictions(sequence,categories):
  predictions = classifier(sequence,categories)[0]
  if not predictions:  # Check if the list of predictions is empty
      return "Unknown"
  max_index = np.argmax([p['score'] for p in predictions])
  max_label = predictions[max_index]["label"]
  return max_label

In [ ]:
from tqdm import tqdm
actual_cats = []
predicted_cats = []

for i in tqdm(range(0,300)):
    try:
        sequence = books.loc[books["simple_categories"]=="Fiction","description"].reset_index(drop=True)[i]
        print(f"Processing sequence {i}: {sequence}")
        predicted_cat = generate_predictions(sequence,fiction_classifier)
        predicted_cats.append(predicted_cat) # Append the predicted_cat directly
        actual_cats.append("Fiction")
    except Exception as e:
        print(f"Error processing sequence {i}: {e}")
        predicted_cats.append("Error")
        actual_cats.append("Fiction")

  0%|          | 0/300 [00:00<?, ?it/s]

Processing sequence 0: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and 


  0%|          | 1/300 [00:03<19:15,  3.86s/it]

Processing sequence 1: A memorable, mesmerizing heroine Jennifer -- brilliant, beautiful, an attorney on the way up until the Mafia's schemes win her the hatred of an implacable enemy -- and a love more destructive than hate. A dangerous, dramatic world The Dark Arena of organized crime and flashbulb lit courtrooms where ambitious prosecutors begin their climb to political power.



  1%|          | 2/300 [00:04<10:45,  2.17s/it]

...


 54%|█████▎    | 161/300 [02:24<01:55,  1.20it/s]

Processing sequence 161: A special anniversary edition of the first publication of the seminal poetic work describes the background of Ginsberg's famous poem, and discusses literary allusions, and techniques of composition used in creating the controversial work. Reprint.



 54%|█████▍    | 162/300 [02:24<01:39,  1.39it/s]

Processing sequence 162: In Coraline's family's new flat there's a locked door. On the other side is a brick wall—until Coraline unlocks the door . . . and finds a passage to another flat in another house just like her own. Only different. The food is better there. Books have pictures that writhe and crawl and shimmer. And there's another mother and father there who want Coraline to be their little girl. They want to change her and keep her with them. . . . Forever. Coraline is an extraordinary fairy tale/nightmare from the uniquely skewed imagination of #1 New York Times bestselling author Neil Gaiman.



  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
for i in tqdm(range(0,300)):
  sequence = books.loc[books["simple_categories"]=="Fiction","description"].reset_index(drop=True)[i]
  predicted_cat = generate_predictions(sequence,fiction_classifier)
  predicted_cats +=[generate_predictions(sequence,fiction_classifier)]
  actual_cats += ["Fiction"]

In [ ]:
prediction_df = pd.DataFrame({"actual_categories":actual_cats,"predicted_categories":predicted_cats})

In [ ]:
prediction_df["correct_prediciton"] = (np.where(prediction_df["actual_categories"]==prediction_df["predicted_categories"],1,0))

In [ ]:
accuracy = prediction_df["correct_prediciton"].sum()/len(prediction_df)
print(f"The accuracy of the model is: {accuracy}")
# The accuracy of the model is: 0.7666666666666667

##### Good accuracy --> goinf to predict all missing

In [ ]:
isbns = []
predicted_cats =[]

missing_cats = books.loc[books["simple_categories"].isna(),["isbn13","description"]].reset_index(drop=True)

In [ ]:
for i in tqdm(range(0,len(missing_cats))):
  sequence = missing_cats["description"][i]
  predicted_cats += [generate_predictions(sequence, fiction_classifier)]
  isbns += [missing_cats["isbn13"][i]]

In [ ]:
missing_predicted_df = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_cats})
books = pd.merge(books,missing_predicted_df,on="isbn13",how="left")
books["simple_categories"] = np.where(books["simple_categories"].isna(),books["predicted_categories"],books["simple_categories"])
books = books.drop(columns=["predicted_categories"])

In [ ]:
books["simple_categories"].isna().sum()

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)
classifier("I love this!")

In [ ]:
classifier(books["description"][0])

In [ ]:
classifier(books["description"][0].split("."))

In [ ]:
# Assuming 'predictions' is a list of dictionaries
print(type(predictions))
print(predictions)
sorted_predictions = sorted(predictions, key=lambda x: x["label"])
sorted_predictions

In [ ]:
def calculate_max_emotion_scores(predictions):
  per_emotion_scores= {label: [] for label in emotion_labels}
  for prediction in predictions:
    sorted_predictions=sorted(prediction,key=lambda x:x["label"])
    for index, label in enumerate(emotion_labels):
      per_emotion_scores[label].append(sorted_predictions[index]["score"])
  return {label: np.max(scores) for label, scores in per_emotion_scores.items()}


In [ ]:
import numpy as np

emotion_labels = ['joy', 'surprise', 'neutral', 'anger', 'sadness', 'disgust', 'fear']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in range(10):
  isbn.append(books["isbn13"][i])
  sentences =books["description"][i].split(".")
  predictions=classifier(sentences)
  max_scores = calculate_max_emotion_scores(predictions)
  for label in emotion_labels:
    emotion_scores[label].append(max_scores[label])

In [ ]:
emotion_scores

In [ ]:
import numpy as np
from tqdm import tqdm

emotion_labels = ['joy', 'surprise', 'neutral', 'anger', 'sadness', 'disgust', 'fear']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}
for i in tqdm(range(len(books))):
  isbn.append(books["isbn13"][i])
  sentences =books["description"][i].split(".")
  predictions=classifier(sentences)
  max_scores = calculate_max_emotion_scores(predictions)
  for label in emotion_labels:
    emotion_scores[label].append(max_scores[label])

In [ ]:
emotions_df=pd.DataFrame(emotion_scores)
emotions_df["isbn13"]=isbn

In [ ]:
emotions_df

In [ ]:
books = pd.merge(books,emotions_df,on="isbn13")

In [ ]:
books

In [ ]:
books.to_csv("books_with_emotions.csv",index=False)